In [8]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [12]:
df = pd.read_csv('Fake.csv')

In [13]:
df.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [14]:
df.shape

(12999, 20)

In [15]:
df.isna().sum()

uuid                     0
ord_in_thread            0
author                2424
published                0
title                  680
text                    46
language                 0
crawled                  0
site_url                 0
country                176
domain_rank           4223
thread_title            12
spam_score               0
main_img_url          3643
replies_count            0
participants_count       0
likes                    0
comments                 0
shares                   0
type                     0
dtype: int64

In [16]:
df = df.fillna(' ')
df.isna().sum()

uuid                  0
ord_in_thread         0
author                0
published             0
title                 0
text                  0
language              0
crawled               0
site_url              0
country               0
domain_rank           0
thread_title          0
spam_score            0
main_img_url          0
replies_count         0
participants_count    0
likes                 0
comments              0
shares                0
type                  0
dtype: int64

In [17]:
df['Req_data'] = df['author'] + df['title']

In [18]:
df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,...,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,Req_data
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,...,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,Barracuda BrigadeMuslims BUSTED: They Stole Mi...
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,...,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,reasoning with factsRe: Why Did Attorney Gener...
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,...,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,Barracuda BrigadeBREAKING: Weiner Cooperating ...
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,...,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,Fed UpPIN DROP SPEECH BY FATHER OF DAUGHTER Ki...
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,Fed UpFANTASTIC! TRUMP'S 7 POINT PLAN To Refor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994,f1b5d0e44803f48732bde854a9fdf95837219b12,2,replaceme,2016-10-26T23:58:00.000+03:00,,It DOES allow you to put a dog face on top of ...,english,2016-10-27T00:37:46.194+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,,40,32,0,0,0,bs,replaceme
12995,36011ceba3647e1bea78299b68b6fb705a1fc1ad,3,Freedumb,2016-10-27T00:02:00.000+03:00,,Wait till you see what happens to the valuatio...,english,2016-10-27T00:37:46.220+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,,40,32,0,0,0,bs,Freedumb
12996,6995d1aa9ac99926106489b14b5530e85358059a,4,major major maj...,2016-10-27T00:06:00.000+03:00,,I'm waiting for the one that puts a pussy on m...,english,2016-10-27T00:37:46.244+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,,40,32,0,0,0,bs,major major maj...
12997,7de8ae90eee164eb756db6c8a3772288e11d7a94,5,beemasters,2016-10-27T00:09:00.000+03:00,,$4 Billion even after they are known to be kee...,english,2016-10-27T00:37:46.247+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,,40,32,0,0,0,bs,beemasters


In [64]:
unique_values = df['type'].unique()
print(unique_values)
print(df['type'][331])

['bias' 'conspiracy' 'fake' 'bs' 'satire' 'hate' 'junksci' 'state']
conspiracy


In [21]:
df['Req_data'][15:20]

15    EdJennerTRUMP SUPPORTER GOT NUTS On MSNBC Repo...
16    Fed UpTOMI LAHREN Has Special Message For Cele...
17    EdJenner#BoycottComedian…ROBERT DENIRO Wanted ...
18    EdJennerHE’S NEVER SOLD AN ORIGINAL PAINTING U...
19    EdJennerSORRY LIBERALS…You Can Stop With The P...
Name: Req_data, dtype: object

In [24]:
p = PorterStemmer()
def stemming(Req_data):
    stm = re.sub('[^a-zA-Z]', " ", Req_data)
    stm = stm.lower()
    stm = stm.split()
    stm = [p.stem(word) for word in stm if not word in stopwords.words('english')]
    stm = " ".join(stm)
    return stm

In [25]:
df['Req_data'] = df['Req_data'].apply(stemming)

In [26]:
df['Req_data']

0        barracuda brigademuslim bust stole million gov...
1        reason factsr attorney gener loretta lynch ple...
2        barracuda brigadebreak weiner cooper fbi hilla...
3        fed uppin drop speech father daughter kidnap k...
4        fed upfantast trump point plan reform healthca...
                               ...                        
12994                                             replacem
12995                                             freedumb
12996                                      major major maj
12997                                              beemast
12998                                               confus
Name: Req_data, Length: 12999, dtype: object

In [92]:
keywords = ['bias', 'conspiracy', 'fake', 'bs', 'hate']
df['label'] = [
    0 if row_type in keywords else 1 
    for row_type in df['type']
]

In [93]:
df['label']

0        0
1        0
2        0
3        0
4        0
        ..
12994    0
12995    0
12996    0
12997    0
12998    0
Name: label, Length: 12999, dtype: int64

In [94]:
x= df['Req_data'].values
y = df['label'].values

In [95]:
print(x)

['barracuda brigademuslim bust stole million gov benefit'
 'reason factsr attorney gener loretta lynch plead fifth'
 'barracuda brigadebreak weiner cooper fbi hillari email investig' ...
 'major major maj' 'beemast' 'confus']


In [96]:
vec = TfidfVectorizer()
vec.fit(x)
x = vec.transform(x)

In [97]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 104904 stored elements and shape (12999, 18822)>
  Coords	Values
  (0, 1699)	0.41068558033381497
  (0, 1854)	0.34317033891362975
  (0, 2340)	0.46265967032547406
  (0, 2560)	0.35372700493374454
  (0, 6936)	0.3741068516538302
  (0, 10713)	0.26779273591111913
  (0, 16100)	0.4033928880999329
  (1, 1262)	0.33442863242615506
  (1, 5851)	0.45332525080876723
  (1, 6066)	0.37872076603462496
  (1, 6639)	0.3005488004789726
  (1, 9787)	0.3599256853107265
  (1, 9905)	0.32779528259136004
  (1, 12608)	0.3631238065868608
  (1, 13489)	0.2829126394346142
  (2, 1699)	0.46344587073357807
  (2, 2338)	0.5220970105476207
  (2, 3664)	0.4145282531356396
  (2, 5436)	0.2407267247202833
  (2, 5976)	0.24560954321319844
  (2, 7690)	0.18660865392455864
  (2, 8400)	0.27518558428190787
  (2, 18137)	0.3346276506827947
  (3, 3324)	0.1755154810794187
  (3, 4080)	0.29456946930344413
  :	:
  (12982, 15613)	1.0
  (12983, 8886)	0.7071067811865476
  (12983, 15536)	

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)


In [99]:
X_train.shape

(10399, 18822)

In [118]:
X_test.shape
print((X_test.data == 1.0).sum())

99


In [101]:
model = LogisticRegression()
model.fit(X_train, Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [102]:
train_predict =model.predict(X_train)
print("train accuracy:", accuracy_score(train_predict, Y_train))

train accuracy: 0.9726896817001635


In [103]:
test_predict =model.predict(X_test)
print("train accuracy:", accuracy_score(test_predict, Y_test))

train accuracy: 0.9726923076923077


In [166]:
data = X_test[2100]
pre = model.predict(data)
if pre[0] == 1:
    print("It is a rumour")
else:
    print("It' is a real news")

It' is a real news
